#Tendencia de calificaciones por género
•	Hipótesis:
 Los géneros con mayor frecuencia muestran variaciones significativas en sus calificaciones (imdb_score) a lo largo de los años, lo que podría indicar cambios en las preferencias de los espectadores o en la calidad del contenido producido.

•	Objetivo:
Identificar los géneros más populares en términos de frecuencia, y analizar sus calificaciones promedio (imdb_score) a través de los años (release_year) para entender cómo han evolucionado en popularidad y calidad.


In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymongo

In [ ]:
# Datos en formato CSV
data_netflix = pd.read_csv("data_netflix.csv")
mejores_shows = pd.read_csv("Mejores_shows_netflix.csv")
mejores_peliculas = pd.read_csv("Mejores_películas_Netflix.csv")
actores = pd.read_csv("Actores.csv")

In [ ]:
# Conexión a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Acceso a la BD
db = client["netflix"]

# Acceso a la colección
collection = db['data_netflix']

In [ ]:
pipeline = [
    #Agrupación por año y género para calcular la moda y el score promedio
  {
    $group: {
      _id: {
        release_year: "$release_year",
        genre: "$genres"
      },
      average_score: { $avg: "$imdb_score" },
      genre_count: { $sum: 1 }
    }
  },
  # Agruparción por año para obtener la moda de géneros más repetidos
  {
    $group: {
      _id: "$_id.release_year",
      genres_stats: {
        $push: {
          genre: "$_id.genre",
          average_score: "$average_score",
          genre_count: "$genre_count"
        }
      }
    }
  },
  #  Obtención del género con la mayor cantidad de repeticiones
  {
    $set: {
      max_genre_count: {
        $max: "$genres_stats.genre_count"
      }
    }
  },
	# Filtro de géneros con el máximo genre_count y generación de una lista
  {
    $set: {
      most_common_genres: {
        $filter: {
          input: "$genres_stats",
          as: "genre",
          cond: { $eq: ["$$genre.genre_count", "$max_genre_count"] }
        }
      }
    }
  },
	{
    $unwind: "$most_common_genres"
  },
	{
        "$replaceRoot": {
            "newRoot": {
                release_year: "$_id",
                genre_frec: "$most_common_genres.genre",
                average_score_genre: "$most_common_genres.average_score",
                genre_count: "$most_common_genres.genre_count"
            }
        }
    },
		{$out: 'year_genre_score'}

]


result = collection.aggregate(pipeline)

